This notebook aims to produce transport layers that are fit for the model.

Transport modes are among: roads, railways, waterways, maritime, and multimodal. Roads are required. Note that the multimodal is required if there are more than 2 transport modes.

### Input
- For each mode, a `raw_<mode>_edges.geojson`
- Required attributes:
    - `raw_roads_edges.geojson`: "class" ("primary", "seconday", etc.), "surface" ("paved, "unpaved")
    - `raw_multimodal_edges.geojson`: "multimodes" ("roads-railways", "roads-maritime", etc.)
- Optional attributes :
    - `raw_<mode>_edges.geojson`: "capacity" (float, max daily tonnage on the edge)
       
### Output
- For each mode, a `<mode>_edges.geojson`
- New attributes :
    - `<mode>_edges.geojson` and `<mode>_nodes.geoson`: 'id' (integer)
    - `<mode>_edges.geojson`: 'id (integer), 'km' (float, length of edge)

In [1]:
region = "ECA"

In [2]:
import os
import time
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import shapely.wkt
from tqdm import tqdm

if region == "Italia":
    input_folder = os.path.join('..', '..', '..', '..', 'Research', 'Elisa', "disruptsc-ita", "input", "Italy", "Transport")
else:
    input_folder = os.path.join('..', '..', '..', region, 'Data', 'Structured', "Transport")

output_folder = os.path.join('..', 'input', region, 'Transport')

In [3]:
projected_crs = {
    'Cambodia': 3857,
    'Ecuador': 31986,
    'ECA': 3857,
    'Italia': 32633
}
projected_crs = projected_crs[region]

In [4]:
def loadShp(filename):
    gpdf = gpd.read_file(filename)
    gpdf = gpdf[~gpdf['geometry'].isnull()]
    gpdf = gpdf.to_crs(epsg=4326)
    return gpdf

In [5]:
def createNodes(df_links):
    all_coords = df_links['geometry'].apply(getEndCoordsFromLine).to_list()
    all_coords = list(set([item for sublist in all_coords for item in sublist]))
    return gpd.GeoDataFrame({"geometry": [Point(coords) for coords in all_coords], "id": range(len(all_coords))}, crs=4326)

def getEndCoordsFromLine(linestring_obj):
    end1Coord = linestring_obj.coords[0]
    end2Coord = linestring_obj.coords[-1]
    return [end1Coord, end2Coord]

def getEndPointsFromLine(linestring_obj):
    end1Coord = linestring_obj.coords[0]
    end2Coord = linestring_obj.coords[-1]
    return Point(*end1Coord), Point(*end2Coord)

def getIndexClosestPoint(point, df_with_points):
    distList = [point.distance(item) for item in df_with_points['geometry'].tolist()]
    return int(df_with_points.index[distList.index(min(distList))])

def updateLineString(linestring_obj, newEnd1, newEnd2):
    return LineString([newEnd1.coords[0]] + linestring_obj.coords[1:-1]+[newEnd2.coords[0]])

def assignEndpointsAndUpdate(df_links, id_link, df_nodes, update=False):
    p1, p2 = getEndPointsFromLine(df_links.loc[id_link, 'geometry'])
    id_closest_point1 = getIndexClosestPoint(p1, df_nodes)
    id_closest_point2 = getIndexClosestPoint(p2, df_nodes)
    df_links.loc[id_link, 'end1'] = id_closest_point1
    df_links.loc[id_link, 'end2'] = id_closest_point2
    if update:
        df_links.loc[id_link, 'geometry'] = updateLineString(df_links.loc[id_link, 'geometry'], df_nodes.loc[id_closest_point1, 'geometry'], df_nodes.loc[id_closest_point2, 'geometry'])
    return df_links

def assignEndpointsAndUpdateFullDf(df_links, df_nodes, update=False):
    print('Assigning end nodes to linestring')
    if update:
        print('Stag liens to endnodes')
    res = df_links.copy()
    for i in tqdm(res.index):
        res = assignEndpointsAndUpdate(res, i, df_nodes, update=update)
    res['end1'] = res['end1'].astype(int)
    res['end2'] = res['end2'].astype(int)
    return res

def getAllEndpoints(df_links):
    all_endpoints = [getEndPointsFromLine(item) for item in df_links['geometry']]
    return [item for sublist in all_endpoints for item in sublist]

def mergePoints(df_nodes, df_links, epsilon):
    print("Nb of original nodes:", df_nodes.shape[0])
    
    all_endpoints = getAllEndpoints(df_links)
    print("Nb of endpoints:", len(all_endpoints))
    
    all_points_gpd = gpd.GeoDataFrame({"geometry": df_nodes['geometry'].tolist()+getAllEndpoints(df_links)}, crs={'epsg':'4326'})
    buffered_polygons = gpd.GeoDataFrame({"geometry": all_points_gpd.buffer(distance=epsilon)}, crs={'epsg':'4326'})
    multipolygon = buffered_polygons.unary_union
    centroids_each_polygon = [polygon.centroid for polygon in multipolygon]
    print("Nb of grouped points:", len(centroids_each_polygon))
    
    return gpd.GeoDataFrame({"id":range(len(multipolygon)), "geometry":centroids_each_polygon}, crs={'epsg':'4326'})

def assignEndpointsOneEdge(row, df_nodes):
    p1, p2 = getEndPointsFromLine(row['geometry'])
    id_closest_point1 = getIndexClosestPoint(p1, df_nodes)
    id_closest_point2 = getIndexClosestPoint(p2, df_nodes)
    row['end1'] = id_closest_point1
    row['end2'] = id_closest_point2
    return row

def assignEndpoints(df_links, df_nodes):
    return df_links.apply(lambda row: assignEndpointsOneEdge(row, df_nodes), axis=1)

In [6]:
def loadAndFormatGeojson(transport_mode, nodeedge, subfolder, suffix=""):
    if nodeedge == "nodes":
        nodes = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_nodes"+suffix+".geojson"))
        if 'index' in nodes.columns:
            nodes = nodes.drop('index', axis=1)
        nodes['id']=range(nodes.shape[0])
        nodes.index = nodes['id']
        nodes.index.name = "index"
        print("There are", nodes.shape[0], "nodes")
        print(nodes.crs)
        return nodes
    
    if nodeedge == "edges":
        edges = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_edges"+suffix+".geojson"))
        edges = edges[~edges['geometry'].isnull()]
        if 'index' in edges.columns:
            edges = edges.drop('index', axis=1)
        edges['id']=range(edges.shape[0])
        edges['end1'] = None
        edges['end2'] = None
        if "capacity" not in edges.columns:
            edges['capacity'] = None
        edges.index = edges['id']
        edges.index.name = "index"
        print("There are", edges.shape[0], "edges")
        print(edges.crs)
        return edges
    
    
def addKm(edges, crs):
    # Project the layer. Watch out, the CRS should be adapted to the country
    edges['km'] = edges.to_crs({'init': 'epsg:'+str(crs)}).length/1000
    return edges

In [7]:
from shapely.ops import linemerge, unary_union
from sklearn.neighbors import BallTree
from shapely.geometry import Point, LineString
from shapely.ops import snap


def export(nodes, edges, input_folder, output_folder, transport_mode, special_suffix):
    subfolder = os.path.join(input_folder, transport_mode.capitalize())
    
    filename = os.path.join(output_folder, transport_mode+"_edges"+special_suffix+".geojson")
    edges.to_file(filename, driver="GeoJSON", index=False)
    print(filename + ' exported')
    
    filename = os.path.join(subfolder, "treated_"+transport_mode+"_edges"+special_suffix+".geojson")
    edges.to_file(filename, driver="GeoJSON", index=False)
    print(filename + ' exported')
    
    filename = os.path.join(subfolder, "treated_"+transport_mode+"_nodes"+special_suffix+".geojson")
    nodes.to_file(filename, driver="GeoJSON", index=False)
    print(filename + ' exported')


def create_nodes_and_update_edges_legacy(edges):
    # create nodes from endpoints
    endpoints = gpd.GeoDataFrame({"end1": edges.geometry.apply(lambda line: Point(line.coords[0])), "end2": edges.geometry.apply(lambda line: Point(line.coords[-1]))})
    all_endpoints = gpd.GeoDataFrame(pd.concat([endpoints['end1'], endpoints['end2']]), columns=["geometry"], crs=edges.crs)
    all_endpoints['geometry_wkt'] = all_endpoints['geometry'].to_wkt()
    nodes = all_endpoints.drop_duplicates('geometry_wkt').copy()
    nodes['id'] = range(nodes.shape[0])
    nodes.index = nodes['id']
    
    # add nodes_id into end1 and end2 columns of edges
    edges['end1'] = endpoints['end1'].to_wkt().map(nodes.set_index('geometry_wkt')['id'])
    edges['end2'] = endpoints['end2'].to_wkt().map(nodes.set_index('geometry_wkt')['id'])

    return nodes, edges


def update_linestring(linestring, new_end1, new_end2):
    return LineString([new_end1.coords[0]] + linestring.coords[1:-1]+[new_end2.coords[0]])


def create_nodes_and_update_edges_legacy(edges):
    """Create unique nodes from endpoints using a spatial tolerance and update edge indices."""
    endpoints = gpd.GeoDataFrame({
        "end1": edges.geometry.apply(lambda line: Point(line.coords[0])),
        "end2": edges.geometry.apply(lambda line: Point(line.coords[-1]))
    })
    
    # Combine all endpoints into one GeoDataFrame
    all_endpoints = gpd.GeoDataFrame(pd.concat([endpoints['end1'], endpoints['end2']]), columns=["geometry"], crs=edges.crs)
    
    # Convert points to numpy array for spatial searching
    endpoint_coords = np.array([[p.x, p.y] for p in all_endpoints.geometry])
    
    # Use BallTree to find unique nodes within the tolerance
    unique_nodes = []
    node_map = {}  # Maps original points to unique node IDs
    
    if len(endpoint_coords) > 0:
        unique_coords = []
        for p in tqdm(endpoint_coords, total=len(endpoint_coords), desc="identifying unique endpoint"):
            x, y = p
            coord = np.array([x, y])
            if unique_coords:
                tree = BallTree(np.array(unique_coords), metric='euclidean')
                dist, nearest_idx = tree.query([coord], k=1, return_distance=True)
                distance = dist[0][0]
                if distance < TOLERANCE:
                    node_id = node_map[tuple(unique_coords[nearest_idx[0][0]])]
                else:
                    node_id = len(unique_nodes)
                    unique_nodes.append(coord)
                    unique_coords.append(coord.tolist())
            else:
                node_id = len(unique_nodes)
                unique_nodes.append(coord)
                unique_coords.append(coord.tolist())
    
            node_map[(x, y)] = node_id
    
    print(f"There are {len(unique_nodes)} unique node for {len(endpoint_coords)} endpoints")
    
    # Create GeoDataFrame for unique nodes
    nodes = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in unique_nodes], crs=edges.crs)
    nodes["id"] = range(len(nodes))
    nodes.index = nodes["id"]
    
    # Map endpoints to their closest node ID
    edges['end1'] = endpoints['end1'].apply(lambda p: node_map.get((p.x, p.y)))
    edges['end2'] = endpoints['end2'].apply(lambda p: node_map.get((p.x, p.y)))
    
    # Update edges' geometry to ensure they start at end1 and end at end2
    print('Update geometries')
    edges['geometry'] = edges.apply(
        lambda row: update_linestring(row.geometry, nodes.loc[row['end1']].geometry, nodes.loc[row['end2']].geometry), axis=1
    )

    return nodes, edges


from shapely import wkt

def create_nodes_and_update_edges(edges):
    """Create unique nodes from endpoints using a spatial tolerance and update edge indices."""
    endpoints = gpd.GeoDataFrame({
        "end1": edges.geometry.apply(lambda line: Point(line.coords[0])),
        "end2": edges.geometry.apply(lambda line: Point(line.coords[-1]))
    })
    
    # Combine all endpoints into one GeoDataFrame
    all_endpoints = gpd.GeoDataFrame(pd.concat([endpoints['end1'], endpoints['end2']]), columns=["geometry"], crs=edges.crs)

    all_endpoints['geometry_wkt'] = all_endpoints['geometry'].apply(lambda geom: wkt.dumps(geom, rounding_precision=5))
    nodes = all_endpoints.drop_duplicates('geometry_wkt').copy()
    nodes['id'] = range(nodes.shape[0])
    nodes.index = nodes['id']
    nodes['long'] = nodes['geometry'].x
    nodes['lat'] = nodes['geometry'].y

    # add nodes_id into end1 and end2 columns of edges
    end1_wkt = endpoints['end1'].apply(lambda geom: wkt.dumps(geom, rounding_precision=5))
    end2_wkt = endpoints['end2'].apply(lambda geom: wkt.dumps(geom, rounding_precision=5))
    edges['end1'] = end1_wkt.map(nodes.set_index('geometry_wkt')['id'])
    edges['end2'] = end2_wkt.map(nodes.set_index('geometry_wkt')['id'])
    
    # Update edges' geometry to ensure they start at end1 and end at end2
    print('Update geometries')
    edges['geometry'] = edges.apply(
        lambda row: update_linestring(row.geometry, nodes.loc[row['end1']].geometry, nodes.loc[row['end2']].geometry), axis=1
    )

    return nodes, edges


# Roads

In [8]:
import re
from tqdm import tqdm


def get_merge_nodes(edges):
    end_points_connectivity = pd.concat([edges['end1'], edges['end2']]).value_counts()
    merge_points = end_points_connectivity[end_points_connectivity == 2].index.sort_values().to_list()
    return merge_points


def get_edges_from_endpoints(edges, endpoints: list):
    merged_df_end1 = pd.merge(pd.DataFrame({'endpoint': endpoints}), edges[['id', 'end1']].rename(columns={'id': 'edge_id', 'end1': 'endpoint'}), on='endpoint', how='left').dropna().astype(int)
    merged_df_end2 = pd.merge(pd.DataFrame({'endpoint': endpoints}), edges[['id', 'end2']].rename(columns={'id': 'edge_id', 'end2': 'endpoint'}), on='endpoint', how='left').dropna().astype(int)
    merged_df = pd.concat([merged_df_end1, merged_df_end2])
    return merged_df.groupby("endpoint")['edge_id'].apply(list).to_dict()


def check_degree2(node_id_to_edges_ids: dict):
    return (pd.Series(node_id_to_edges_ids).apply(len) == 2).all()


def merge_lines_with_tolerance(lines):
    """Snap lines to each other within a tolerance and then merge them."""
    snapped_lines = [snap(line, lines[0], TOLERANCE) for line in lines]  # Snap all lines to the first one
    return linemerge(snapped_lines)  # Merge the snapped lines


def merge_edges_attributes(gdf):
    new_data = {}
    new_data['geometry'] = linemerge(gdf.geometry.to_list())
    if new_data['geometry'].geom_type != "LineString":
        print(gdf.geometry)
        print(new_data['geometry'])
        raise ValueError(f"Merged geometry is: {new_data['geometry'].geom_type}")

    def string_to_list(s):
        return list(map(int, re.findall(r'\d+', s)))
    def merge_or_unique(column):
        unique_vals = gdf[column].dropna().unique()
        return unique_vals[0] if len(unique_vals) == 1 else ', '.join(unique_vals)
        
    # Aggregate columns based on given rules
    if 'km' in gdf.columns:
        new_data['km'] = gdf['km'].sum()
    if 'osmids' in gdf.columns:
        new_data['osmids'] = str(string_to_list(', '.join(map(str, gdf['osmids'].fillna('')))))
    if 'name' in gdf.columns:
        new_data['name'] = ', '.join(filter(None, gdf['name'].astype(str)))  # Ignore None values
    if 'capacity' in gdf.columns:
        new_data['capacity'] = gdf['capacity'].min()
    for col in ['end1', 'end2']:
        if col in gdf.columns:
            new_data[col] = None
    for col in ['special', 'class', 'surface', 'disruption']:
        if col in gdf.columns:
            new_data[col] = merge_or_unique(col)

    # Create a new row with the merged data
    return new_data

def update_gdf(gdf, new_data, old_ids, new_id):
    for col, value in new_data.items():
        gdf.at[new_id, col] = value
    gdf.loc[list(set(old_ids) - {new_id}), 'to_keep'] = False


def update_dict(my_dict, old_value, new_value):
    for key, value_list in my_dict.items():
        if old_value in value_list:
            my_dict[key] = [new_value if v == old_value else v for v in value_list]
    return my_dict


def remove_degree_2_nodes(edges):
    merge_nodes = get_merge_nodes(edges)
    print(f"Nb degree 2 nodes: {len(merge_nodes)}")
    merge_nodes_with_edges_ids = get_edges_from_endpoints(edges, merge_nodes)
    print(f"Check that they are actually 2 edges associated: {check_degree2(merge_nodes_with_edges_ids)}")
    
    edges['to_keep'] = True
    edges = edges.set_index('id')
    for merged_node in tqdm(list(merge_nodes_with_edges_ids.keys()), total=len(merge_nodes_with_edges_ids),
                           desc="merging edges"):
        edge_ids = merge_nodes_with_edges_ids.pop(merged_node)
        if edge_ids[0] == edge_ids[1]:
            print(f'One self loop generated {edge_ids[0]}')
            continue
        merged_attributes = merge_edges_attributes(edges.loc[edge_ids])
        old_id = max(edge_ids)
        new_id = min(edge_ids)
        update_gdf(edges, merged_attributes, edge_ids, new_id)
        merge_nodes_with_edges_ids = update_dict(merge_nodes_with_edges_ids, old_id, new_id)
    
    print(f"Check that all resulting geometries are LineString: {edges['geometry'].apply(lambda geom: geom.geom_type == 'LineString').all()}")
    
    edges = edges[edges['to_keep']]
    edges = edges.drop(columns=['to_keep'])
    edges = edges.reset_index()
    return edges

#s = remove_degree_2_nodes(edges)

In [9]:
import geopandas as gpd
import pandas as pd
from shapely import Point
from shapely.ops import split, snap
from shapely.geometry import MultiPoint
from tqdm import tqdm

TOLERANCE = 0.0001


def geometry_to_list_of_single_geom(geom, target_type: str):
    if (geom.geom_type == "Linestring") and (target_type == "LineString"):
        return [geom]
    if (geom.geom_type == "Point") and (target_type == "Point"):
        return [geom]
    if (geom.geom_type == "MultiLineString") and (target_type == "LineString"):
        return [single_geom for single_geom in geom.geoms]
    if (geom.geom_type == "MultiPoint") and (target_type == "Point"):
        return [single_geom for single_geom in geom.geoms]
    if geom.geom_type == "GeometryCollection":
        geoms = [single_geom for single_geom in geom.geoms]
        list_of_single_geoms = [geometry_to_list_of_single_geom(single_geom, target_type) for single_geom in geoms]
        return [item for sublist in list_of_single_geoms for item in sublist]
    return []


def treat_overlapping_linestrings(line0, line1):
    if line0.within(line1):
        return None, [line1]
    elif line1.within(line0):
        return [line0], None
    elif line0.overlaps(line1):
        overlapping_parts = geometry_to_list_of_single_geom(line0.intersection(line1), "LineString")
        new_line0 = overlapping_parts
        remaining_geom = line0.difference(line1)
        if remaining_geom.geom_type == "LineString":
            new_line1 = [remaining_geom]
        elif remaining_geom.geom_type == "MultiLineString":
            new_line1 = [geom for geom in remaining_geom.geoms]
        return new_line0, new_line1
    else:
        return False


def extract_endpoints(line):
    return [Point(line.coords[0]), Point(line.coords[-1])]


def get_intersection_without_endpoints(line0, line1):
    intersection = line0.intersection(line1)
    if intersection.is_empty:
        return False
    intersection_points = []
    if intersection.geom_type == "Point":  # Single intersection point
        intersection_points += [intersection]
    elif intersection.geom_type == "MultiPoint":  # Multiple intersection points
        intersection_points = list(intersection.geoms)
    elif intersection.geom_type == "GeometryCollection":  # super rare
        intersection_points += geometry_to_list_of_single_geom(intersection, "Point")
        intersection_points += geometry_to_list_of_single_geom(intersection, "MultiPoint")
        linestrings = geometry_to_list_of_single_geom(intersection, "LineString")
        intersection_points += [Point(coord) for linestring in linestrings for coord in linestring.coords]
    elif intersection.geom_type == "LineString":  # super rare
        intersection_points = [Point(coord) for coord in intersection.coords]
    elif intersection.geom_type == "MultiLineString":  # super rare
        linestrings = geometry_to_list_of_single_geom(intersection, "LineString")
        intersection_points = [Point(coord) for linestring in linestrings for coord in linestring.coords]
    else:
        print(line0, line1, intersection)
        raise ValueError(str(intersection.geom_type))
    endpoints = extract_endpoints(line0) + extract_endpoints(line1)
    intersection_points = [point for point in intersection_points if not is_close_to_any(point, endpoints)]
    if len(intersection_points) == 0:
        return False
    else:
        return list(set(intersection_points))


def is_close_to_any(point, point_list):
    """Check if a point is close to any endpoint using shapely's snap function."""
    return any(snap(point, p, TOLERANCE).equals(p) for p in point_list)


def find_new_geometries_for_overlapping_edges(edges, specific_ids=False):
    # Build a spatial index
    spatial_index = edges.sindex

    # Identify LineStrings whose coordinates are fully within another LineString
    # cond_fully_within = pd.Series(False, index=edges.index)
    new_geometries = {}  # index: new geom
    chunck_to_process = edges
    if isinstance(specific_ids, list):
        chunck_to_process = edges.loc[specific_ids]
    for i, row1 in tqdm(chunck_to_process.iterrows(), total=len(chunck_to_process),
                        desc="Processing overlapping edges"):
        if i in new_geometries.keys():
            continue
        # Use the spatial index to find potential candidates
        possible_matches_index = list(spatial_index.intersection(row1.geometry.bounds))
        possible_matches = edges.iloc[possible_matches_index]

        # Compare only with potential candidates
        for j, row2 in possible_matches.iterrows():
            if (i != j) and (j not in new_geometries.keys()):
                new_geoms = treat_overlapping_linestrings(row1.geometry, row2.geometry)
                if new_geoms:
                    new_geometries[i], new_geometries[j] = new_geoms
    return new_geometries


def format_new_edges(edges, new_geometries):
    new_edges_list = []
    for i, new_geom in new_geometries.items():
        if new_geom:
            new_edges = gpd.GeoDataFrame(geometry=new_geometries[i], crs=edges.crs)
            row_data = edges.loc[[i]].drop(columns=["geometry"])  # Keep original structure
            row_data_repeated = row_data.loc[row_data.index.repeat(len(new_geometries[i]))]  # Repeat for each part
            new_edges = pd.concat([new_edges, row_data_repeated.reset_index(drop=True)], axis=1)
            new_edges_list += [new_edges]
    all_new_edges = pd.concat(new_edges_list)
    all_new_edges = all_new_edges[~all_new_edges['geometry'].isnull()]
    return all_new_edges


def add_new_edges_to_gdf(edges, new_geometries, all_new_edges):
    print(f'Removing {len(new_geometries.keys())} edges due to the removal of overlapping parts')
    edges = edges.drop(index=list(new_geometries.keys()))
    print(f'Adding {all_new_edges.shape[0]} edges')
    edges = pd.concat([edges, all_new_edges], ignore_index=True)

    edges['id'] = list(range(edges.shape[0]))
    edges.index = edges['id']
    new_ids = edges['id'].iloc[-all_new_edges.shape[0]:].to_list()

    return edges, new_ids


def treat_overlapping_edges(edges, specific_ids=False):
    new_geometries = find_new_geometries_for_overlapping_edges(edges, specific_ids)
    if len(new_geometries) > 0:
        all_new_edges = format_new_edges(edges, new_geometries)
        return add_new_edges_to_gdf(edges, new_geometries, all_new_edges)
    else:
        return edges, []


def treat_intersecting_edges(edges, specific_ids=False):
    new_geometries = find_splitted_lines(edges, specific_ids)
    if len(new_geometries) > 0:
        all_new_edges = format_new_edges(edges, new_geometries)
        return add_new_edges_to_gdf(edges, new_geometries, all_new_edges)
    else:
        return edges, []


def find_splitted_lines(edges, specific_ids=False):
    # Build a spatial index
    spatial_index = edges.sindex

    # Identify LineStrings whose coordinates are fully within another LineString
    new_geometries = {}  # index: new geom
    chunck_to_process = edges['geometry'].to_dict()
    if isinstance(specific_ids, list):
        chunck_to_process = edges.loc[specific_ids, 'geometry'].to_dict()
    for i, row1_geom in tqdm(chunck_to_process.items(), total=len(chunck_to_process),
                             desc="Processing intersecting edges"):
        if i in new_geometries.keys():
            continue
        # Use the spatial index to find potential candidates
        possible_matches_index = list(spatial_index.intersection(row1_geom.bounds))
        possible_matches = edges.iloc[possible_matches_index]['geometry'].to_dict()

        # Compare only with potential candidates
        for j, row2_geom in possible_matches.items():
            if (i != j) and (j not in new_geometries.keys()):
                # print(i, j)
                new_intersection_points = get_intersection_without_endpoints(row1_geom, row2_geom)
                if new_intersection_points:
                    #print(i, j, get_intersection_without_endpoints(row1_geom, row2_geom))
                    #print(chunck_to_process[i], chunck_to_process[j])
                    split_geometry_collection1 = split(row1_geom, MultiPoint(new_intersection_points))
                    split_geometry_collection2 = split(row2_geom, MultiPoint(new_intersection_points))
                    new_geometries[i] = [geom for geom in split_geometry_collection1.geoms]
                    new_geometries[j] = [geom for geom in split_geometry_collection2.geoms]

    return new_geometries

In [10]:
i, j = 309, 380
print(edges.loc[[i, j]])
ax = edges.loc[[i, j]].plot()
line0 = edges.loc[i,'geometry']
line1 = edges.loc[j,'geometry']
inter = line0.intersection(line1)
print(inter)
gpd.GeoSeries([inter]).plot(ax=ax, color='red', markersize=50)
print(is_close_to_any(inter, extract_endpoints(line0) + extract_endpoints(line1)))
get_intersection_without_endpoints(line0, line1)

NameError: name 'edges' is not defined

In [ ]:
def remove_self_loop(edges):
    # remove selfloop
    cond = edges['end1'] == edges['end2']
    if cond.sum() > 0:
        print(f"Removing {cond.sum()} self-loops")
        edges = edges[~cond]
    return edges

def remove_duplicated_geometry(edges):
    cond = edges['geometry'].to_wkt().duplicated()
    if cond.sum() > 0:
        print(f"Removing {cond.sum()} duplicated edges")
        edges = edges[~cond]
    return edges

def keep_one_edge_same_endpoints(edges):
    # if several edges have the same start and end points, keep only one
    edges['end_set'] = edges.apply(lambda row: frozenset([row['end1'], row['end2']]), axis=1)  # Create a set representation of each row's end1 and end2
    cond = edges['end_set'].duplicated()
    if cond.sum() > 0:
        print(f"Removing {cond.sum()} edges that have the same endpoints")
        edges = edges[~cond]
    edges = edges.drop(columns=['end_set'])  # Drop the temporary column
    edges['id'] = list(range(edges.shape[0]))
    edges.index = edges['id']
    return edges

In [11]:
transport_mode = 'roads'
special_suffix = "_osmsimp"  # _ximena leave empty "" otherwise
subfolder = os.path.join(input_folder, transport_mode.capitalize())
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder, special_suffix)

There are 15294 edges
EPSG:4326


In [28]:
filename = os.path.join('..', '..', '..', region, 'Data', 'Structured', "Transport", 'roads_edges_osmsimp.geojson')
edges = gpd.read_file(filename)
edges

,id,osmids,special,end1,end2,capacity,surface,class,disruption,name,km,geometry
0,0,"(24745, 24748, 24750, 24754, 16821, 16822, 247...",None,0,1,None,paved,None,None,None,10.039223,"LINESTRING (44.86745 40.6866, 44.85309 40.6969..."
1,1,"[ 86074, 86071, 85944, 86073, 86072, 47751, 18...",,0,6978,None,paved,,,"None, None",15.276297,"LINESTRING (44.86745 40.6866, 44.88625 40.6697..."
2,2,"(11552, 11554, 25476, 74342, 25511, 74348, 194...",None,1,3,None,paved,None,None,None,7.279171,"LINESTRING (44.86559 40.73822, 44.86844 40.745..."
3,3,"(40964, 40967, 40498, 40503, 24126, 12871, 128...",None,2,1,None,paved,None,None,None,44.547590,"LINESTRING (44.50912 40.79478, 44.51165 40.793..."
4,4,"(6147, 6148, 6149, 6152, 6153, 6159, 6169, 617...",None,3,6979,None,paved,None,None,None,6.663715,"LINESTRING (44.9159 40.76316, 44.92095 40.7617..."
...,...,...,...,...,...,...,...,...,...,...,...,...
12040,12040,"(31904, 31905, 31906, 31907, 31910, 31898, 318...",border,776,780,None,paved,None,None,None,0.055694,"LINESTRING (50.01164 40.58368, 50.01214 40.58366)"
12041,12041,"(31904, 31905, 31906, 31907, 31910, 31898, 318...",None,776,780,None,paved,None,None,None,4.871903,"LINESTRING (50.01214 40.58366, 50.01632 40.583..."
12042,12042,"[(38688, 38689, 38691, 16294, 29102, 29107, 29...",None,3898,3857,None,paved,None,None,None,18.293820,"LINESTRING (70.81944 40.06056, 70.82234 40.061..."
12043,12043,"[(38688, 38689, 38691, 16294, 29102, 29107, 29...",border,3898,3857,None,paved,None,None,None,0.063481,"LINESTRING (70.93277 40.15056, 70.93315 40.15088)"


In [35]:
def get_isolated_edges(edges):
    degree_1_nodes = get_degree_n_nodes(edges, 1)
    degree_1_nodes_with_edge_id = get_edges_from_endpoints(edges, degree_1_nodes)
    nb_degree_1_nodes_per_edge_id = pd.Series(degree_1_nodes_with_edge_id).apply(lambda l: l[0]).value_counts()
    edge_ids_with_2_degree_1_endpoints = nb_degree_1_nodes_per_edge_id[nb_degree_1_nodes_per_edge_id == 2]
    return edge_ids_with_2_degree_1_endpoints.index.to_list()

def remove_isolated_edges(edges):
    isolated_edges = get_isolated_edges(edges)
    print(f"Nb isolated edge: {len(isolated_edges)}")
    edges = edges[~edges['id'].isin(isolated_edges)].copy()
    edges['id'] = range(edges.shape[0])
    edges.index = edges['id']
    return edges


def get_degree_n_nodes(edges, n: int):
    end_points_connectivity = pd.concat([edges['end1'], edges['end2']]).value_counts()
    return end_points_connectivity[end_points_connectivity == n].index.sort_values().to_list()


def get_edges_from_endpoints(edges, endpoints: list):
    merged_df_end1 = pd.merge(pd.DataFrame({'endpoint': endpoints}),
                              edges[['id', 'end1']].rename(columns={'id': 'edge_id', 'end1': 'endpoint'}),
                              on='endpoint', how='left').dropna().astype(int)
    merged_df_end2 = pd.merge(pd.DataFrame({'endpoint': endpoints}),
                              edges[['id', 'end2']].rename(columns={'id': 'edge_id', 'end2': 'endpoint'}),
                              on='endpoint', how='left').dropna().astype(int)
    merged_df = pd.concat([merged_df_end1, merged_df_end2])
    return merged_df.groupby("endpoint")['edge_id'].apply(list).to_dict()


In [15]:
country_boundaries_filename = os.path.join('..', '..', '..', region, 'Data', 'Structured', "Admin", 'gadm41_country_boundaries.geojson')
country_boundaries = gpd.read_file(country_boundaries_filename)
borders = country_boundaries.union_all().boundary

In [443]:
transport_mode = 'roads'
special_suffix = "_osmsimp"  # _ximena leave empty "" otherwise
subfolder = os.path.join(input_folder, transport_mode.capitalize())
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder, special_suffix)

print("Assigning nodes")
nodes, edges = create_nodes_and_update_edges(edges)

edges = remove_self_loop(edges)
edges = remove_duplicated_geometry(edges)
edges = keep_one_edge_same_endpoints(edges)

edges, new_ids = treat_overlapping_edges(edges)
edges, new_ids = treat_overlapping_edges(edges, new_ids)
print(f"Check that all resulting geometries are LineString: {edges['geometry'].apply(lambda geom: geom.geom_type == 'LineString').all()}")

edges, new_ids = treat_intersecting_edges(edges)
edges, new_ids = treat_intersecting_edges(edges, new_ids)
print(f"Check that all resulting geometries are LineString: {edges['geometry'].apply(lambda geom: geom.geom_type == 'LineString').all()}")

# add columns
edges['surface'] = 'paved'
for col in ['surface', 'class', 'disruption', 'name', 'special']:
    if col not in edges.columns:
        edges[col] = None

print("Assigning nodes")
nodes, edges = create_nodes_and_update_edges(edges)
edges = remove_degree_2_nodes(edges)
edges = remove_self_loop(edges)
edges = keep_one_edge_same_endpoints(edges)
edges = addKm(edges, projected_crs)
#edges.to_file('test.geojson', driver="GeoJSON", index=False)

print(f"There are {edges.shape[0]} edges")
print(edges.head())

# Exports
#export(nodes, edges, input_folder, output_folder, transport_mode, special_suffix)

There are 15294 edges
EPSG:4326
Assigning nodes


identifying unique endpoint: 100%|██████████████████████████████████████████████| 30588/30588 [03:05<00:00, 164.73it/s]


There are 10855 unique node for 30588 endpoints
Update geometries
Removing 38 self-loops
Removing 92 duplicated edges
Removing 197 edges that have the same endpoints


Processing overlapping edges: 100%|█████████████████████████████████████████████| 14967/14967 [00:25<00:00, 577.58it/s]


Removing 88 edges due to the removal of overlapping parts
Adding 68 edges


...

identifying unique endpoint: 100%|██████████████████████████████████████████████| 29924/29924 [03:11<00:00, 156.38it/s]


There are 10922 unique node for 29924 endpoints
Update geometries
Nb degree 2 nodes: 3028
Check that they are actually 2 edges associated: True


merging edges: 100%|██████████████████████████████████████████████████████████████| 3028/3028 [00:12<00:00, 246.97it/s]


Check that all resulting geometries are LineString: True
Removing 2 edges that have the same endpoints
There are 11932 edges
    id                                             osmids special  \
id                                                                  
0    0  (24745, 24748, 24750, 24754, 16821, 16822, 247...    None   
1    1  [86074, 86071, 85944, 86073, 86072, 47751, 180...           
2    2  (78727, 78728, 78616, 78632, 78638, 78643, 297...    None   
3    3  (11552, 11554, 25476, 74342, 25511, 74348, 194...    None   
4    4  (40964, 40967, 40498, 40503, 24126, 12871, 128...    None   

                                             geometry  end1  end2 capacity  \
id                                                                           
0   LINESTRING (44.86745 40.6866, 44.85309 40.6969...   0.0   2.0     None   
1   LINESTRING (44.86745 40.6866, 44.88625 40.6697...   NaN   NaN      NaN   
2   LINESTRING (44.95259 40.61724, 44.95157 40.618...   1.0   0.0     None   
3

C:\Users\Celian\miniforge3\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [444]:
export(nodes, edges, input_folder, output_folder, transport_mode, special_suffix)

..\input\ECA\Transport\roads_edges_osmsimp.geojson exported
..\..\..\ECA\Data\Structured\Transport\Roads\treated_roads_edges_osmsimp.geojson exported
..\..\..\ECA\Data\Structured\Transport\Roads\treated_roads_nodes_osmsimp.geojson exported


# Maritime

In [43]:
transport_mode = 'maritime'
special_suffix = "_mc"  # _ximena leave empty "" otherwise
subfolder = os.path.join(input_folder, transport_mode.capitalize())
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder, special_suffix)

# create nodes from endpoints
endpoints = gpd.GeoDataFrame({"end1": edges.geometry.apply(lambda line: Point(line.coords[0])), "end2": edges.geometry.apply(lambda line: Point(line.coords[-1]))})
unique_endpoints = list(set(endpoints['end1'].to_list()) | set(endpoints['end2'].to_list()))
nodes = gpd.GeoDataFrame(geometry=unique_endpoints, crs=edges.crs)
nodes['id'] = range(nodes.shape[0])

# add nodes_id into end1 and end2 columns of edges
edges['end1'] = endpoints['end1'].map(nodes.set_index('geometry')['id'])
edges['end2'] = endpoints['end2'].map(nodes.set_index('geometry')['id'])

edges = addKm(edges, crs=3975) #for maritime we use 3975, which is projection for the whole world

print(nodes.head())
print(edges.head())

export(nodes, edges, input_folder, output_folder, transport_mode, special_suffix)

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 112 edges
EPSG:4326
                    geometry  id
0        POINT (51 13.00001)   0
1          POINT (19.8 38.6)   1
2  POINT (33.75001 27.89998)   2
3          POINT (32.6 29.7)   3
4      POINT (41.2 16.29999)   4
      from_infra  to_infra    distance    length  id            km capacity  \
index                                                                         
0           None      None         NaN       NaN   0   7223.033494     None   
1           None      None         NaN       NaN   1   6337.468767     None   
2           None      None         NaN       NaN   2  24857.328586     None   
3           None      None         NaN       NaN   3   6546.422559     None   
4       maritime  maritime  532.647058  5.009238   4    585.160733     None   

       end1  end2 special  name  \
index                             
0        42    77    None  None   
1        77    62    None  None   
2        61    77    None  None   
3        61    82    None  None   
4       

C:\Users\Celian\miniforge3\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# Airways

In [19]:
transport_mode = 'airways'
subfolder = os.path.join(input_folder, transport_mode.capitalize())

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)

edges = addKm(edges, projected_crs) #for maritime we use 3975, which is projection for the whole world
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON", index=False)
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON", index=False)

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 3 nodes
epsg:4326
There are 3 edges
epsg:4326


C:\Users\colon\Anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Assigning end nodes to linestring
                                         name special  \
index                                                   
0      Aeropuerto Ecologico Galapagos Seymour    None   
1                        Aeropuerto Guayaquil    None   
2             Aeropuerto Quito Mariscal Sucre    None   

                         geometry  id  
index                                  
0      POINT (-90.26504 -0.45506)   0  
1      POINT (-79.88713 -2.15896)   1  
2      POINT (-78.35621 -0.12718)   2  
      special capacity                                           geometry  id  \
index                                                                           
0        None     None  LINESTRING (-90.26504 -0.45506, -78.35621 -0.1...   0   
1        None     None  LINESTRING (-78.35621 -0.12718, -79.88713 -2.1...   1   
2        None     None  LINESTRING (-79.88713 -2.15896, -90.26504 -0.4...   2   

       end1  end2           km  
index                           
0       

# Waterways

In [11]:
transport_mode = 'waterways'
subfolder = os.path.join(input_folder, transport_mode.capitalize())

nodes = loadAndFormatGeojson(transport_mode, "nodes", subfolder)
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
edges = addKm(edges, projected_crs)
edges = assignEndpointsAndUpdateFullDf(edges, nodes)

print(nodes.head())
print(edges.head())

nodes.to_file(os.path.join(output_folder, transport_mode+"_nodes.geojson"), driver="GeoJSON", index=False)
edges.to_file(os.path.join(output_folder, transport_mode+"_edges.geojson"), driver="GeoJSON", index=False)

if (edges['end1'] == edges['end2']).any():
    print('ATT')

There are 22 nodes
epsg:4326
There are 20 edges
epsg:4326
Assigning end nodes to linestring


C:\Users\colon\Anaconda3\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1250.87it/s]


       id                      name                    geometry
index                                                          
0       0          Chong Kneas Port  POINT (103.82202 13.26983)
1       1      Kampong Chlnang Port  POINT (104.68121 12.26825)
2       2  Kampong Chlnang Junction  POINT (104.69201 12.27107)
3       3         Kratie River Port  POINT (106.01621 12.48460)
4       4           Kratie Junction  POINT (106.01220 12.48335)
       end1  end2 special  capacity  id  \
index                                     
0         2     0    None  200000.0   0   
1        10     6    None   20000.0   1   
2         6     5    None   20000.0   2   
3         6     4    None   20000.0   3   
4         4     3    None       NaN   4   

                                                geometry          km  
index                                                                 
0      LINESTRING (104.69187 12.27092, 104.67544 12.3...  156.544324  
1      LINESTRING (104.95104 11.55619

# Multimodality

In [13]:
def find_anchor_points(nodes, mode):
    if mode == 'roads':
        return nodes
    elif mode == 'maritime':
        return nodes[nodes['port']]
    elif mode == 'railways':
        return nodes[nodes['station']]
    elif mode == 'airways':
        return nodes[nodes['airport']]
    elif mode == 'waterways':
        return nodes[nodes['port']]
    else:
        raise ValueError("Wrong mode choosen")


def build_multimodal_links(from_nodes, from_mode, to_nodes, to_mode):
    anchor_from_nodes = find_anchor_points(from_nodes, from_mode)
    anchor_to_nodes = find_anchor_points(to_nodes, to_mode)

    # Find the closest road for each port
    links = []
    projected_anchor_from_nodes = anchor_from_nodes.to_crs(epsg=3857)
    projected_anchor_to_nodes = anchor_to_nodes.to_crs(epsg=3857)
    for _, target_node in projected_anchor_to_nodes.iterrows():
        # Find the closest road point to the current port
        closest_from_nodes = projected_anchor_from_nodes.distance(target_node.geometry).idxmin()
        closest_point = projected_anchor_from_nodes.loc[closest_from_nodes].geometry
    
        # Create a LineString from port to closest road point
        link = LineString([target_node.geometry, closest_point])
        links.append({"geometry": link})
    
    # Create a new GeoDataFrame for the links
    links_gdf = gpd.GeoDataFrame(links, crs=projected_anchor_to_nodes.crs)
    links_gdf = links_gdf.to_crs(epsg=4326)
    links_gdf['multimodes'] = from_mode + '-' + to_mode
    return links_gdf

In [14]:
# automaed creationg of multimodal split
import geopandas as gpd
from shapely.geometry import LineString

multimode = ["roads", "maritime"]
suffix0 = "_osmsimp"
suffix1 = "_mc"

roads_nodes = gpd.read_file(os.path.join(input_folder, "Roads", "treated_roads_nodes_osmsimp.geojson"))
maritime_nodes = gpd.read_file(os.path.join(input_folder, "Maritime", "treated_maritime_nodes_mc.geojson"))
maritime_nodes['port'] = maritime_nodes['port'].map(lambda x: bool(x) if pd.notna(x) else False)
railways_nodes = gpd.read_file(os.path.join(input_folder, "Railways", "treated_railways_nodes.geojson"))
railways_nodes['station'] = railways_nodes['station'].map(lambda x: bool(x) if pd.notna(x) else False)

multimodal_edges = pd.concat([
    build_multimodal_links(roads_nodes, "roads", maritime_nodes, "maritime"),
    build_multimodal_links(roads_nodes, "roads", railways_nodes, "railways"),
    build_multimodal_links(railways_nodes, "railways", maritime_nodes, "maritime")
])
multimodal_edges['km'] = 0.1  # no impact
multimodal_edges['id'] = range(multimodal_edges.shape[0])
multimodal_edges['capacity'] = None
multimodal_edges.to_file(os.path.join(output_folder, "multimodal_edges_osmsimp.geojson"), driver="GeoJSON", index=False)

print(multimodal_edges.head())
print(multimodal_edges.shape)

                                            geometry      multimodes   km  id  \
0   LINESTRING (41.66864 41.6486, 41.66877 41.64588)  roads-maritime  0.1   0   
1  LINESTRING (51.22857 43.60237, 51.22302 43.60393)  roads-maritime  0.1   1   
2  LINESTRING (41.65711 42.15927, 41.66985 42.16787)  roads-maritime  0.1   2   
3  LINESTRING (49.73252 40.28461, 49.71417 40.28648)  roads-maritime  0.1   3   
0  LINESTRING (71.39464 42.94935, 71.37812 42.94671)  roads-railways  0.1   4   

  capacity  
0     None  
1     None  
2     None  
3     None  
0     None  
(38, 5)


# Railways

In [72]:
transport_mode = 'railways'

subfolder = os.path.join(input_folder, transport_mode.capitalize())
edges = loadAndFormatGeojson(transport_mode, "edges", subfolder)
nodes, edges = create_nodes_and_update_edges(edges)

edges = addKm(edges, projected_crs)
edges['disruption'] = None

print(nodes.head())
print(edges.head())

if (edges['end1'] == edges['end2']).any():
    print('ATT')

export(nodes, edges, input_folder, output_folder, transport_mode, special_suffix="")

There are 31 edges
EPSG:4326
                    geometry  id
0  POINT (71.39464 42.94935)   0
1   POINT (45.2105 41.37973)   1
2  POINT (44.87981 41.65591)   2
3  POINT (82.60064 45.07896)   3
4  POINT (43.64281 40.74461)   4
      OBJECTID NAME1 NAME2 NAME3 ISO_CC RR_FEATURE Shape_Length special  \
index                                                                     
0         None  None  None  None   None       None         None    None   
1         None  None  None  None   None       None         None    None   
2         None  None  None  None   None       None         None    None   
3         None  None  None  None   None       None         None    None   
4         None  None  None  None   None       None         None    None   

                                                geometry  id  end1  end2  \
index                                                                      
0      LINESTRING (51.22432 43.59837, 51.22589 43.606...   0    33    27   
1      LINESTRING (

C:\Users\Celian\miniforge3\Lib\site-packages\pyproj\crs\crs.py:143: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


# Other stuff, may be useful

### Transition from shp to geojson

In [47]:
for edge_node in ["node", "edge"]:    
    transport_mode = 'roads'
    version = "v8"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)

    filename = "raw_"+transport_mode+"_"+edge_node+"s.shp"
    df = gpd.read_file(os.path.join(subfolder, filename))

    version = "current_version"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)
    df.to_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"), driver="GeoJSON")

### Change to CRS 4326

In [76]:
for edge_node in ["node", "edge"]:    
    transport_mode = 'multimodal'
    version = "current_version"
    subfolder = os.path.join(folder, 'Data', "Structured", transport_mode.capitalize(), version)
    
    filename = "raw_"+transport_mode+"_"+edge_node+"s.shp"
    df = gpd.read_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"))
    
    df = df.to_crs(4326)
    
    df.to_file(os.path.join(subfolder, "raw_"+transport_mode+"_"+edge_node+"s.geojson"), driver="GeoJSON")